## ORIGINALNA 

In [1]:
import numpy as np

def evaluate_state_connect4(board, player):
    """
    Evaluate the state of a Connect 4 board for a given player.
    
    Parameters:
    - board: 2D numpy array representing the board. Each cell contains 'W' (white), 'B' (black), or '.' (empty).
    - player: The player for whom the evaluation is being calculated ('W' or 'B').
    
    Returns:
    - A heuristic score for the board state.
    """
    opponent = 'W' if player == 'B' else 'B'
    rows, cols = board.shape
    
    def count_sequences(b, length, p):
        """
        Count sequences of length `length` for player `p` on the board.
        """
        count = 0
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]  # Right, Down, Diagonal Down-Right, Diagonal Down-Left
        for r in range(rows):
            for c in range(cols):
                for dr, dc in directions:
                    sequence = []
                    for step in range(length):
                        nr, nc = r + dr * step, c + dc * step
                        if 0 <= nr < rows and 0 <= nc < cols:
                            sequence.append(b[nr, nc])
                        else:
                            break
                    if len(sequence) == length and sequence.count(p) == length:
                        count += 1
        return count

    def count_threats(b, p):
        """
        Count threats (3 in a row with one empty space) for player `p`.
        """
        count = 0
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
        for r in range(rows):
            for c in range(cols):
                for dr, dc in directions:
                    sequence = []
                    for step in range(4):
                        nr, nc = r + dr * step, c + dc * step
                        if 0 <= nr < rows and 0 <= nc < cols:
                            sequence.append(b[nr, nc])
                        else:
                            break
                    if len(sequence) == 4 and sequence.count(p) == 3 and sequence.count('.') == 1:
                        count += 1
        return count

    # 1. Winning positions
    player_wins = count_sequences(board, 4, player)
    opponent_wins = count_sequences(board, 4, opponent)
    
    if player_wins > 0:
        return float('inf')  # Player has won.
    if opponent_wins > 0:
        return float('-inf')  # Opponent has won.

    # 2. Threats
    player_threats = count_threats(board, player)
    opponent_threats = count_threats(board, opponent)

    # 3. Positional control (center column preference)
    center_col = cols // 2
    center_score = sum(1 for r in range(rows) if board[r, center_col] == player) - \
                   sum(1 for r in range(rows) if board[r, center_col] == opponent)
    
    # 4. Mobility (empty spaces in accessible positions)
    player_mobility = sum(
        1 for r in range(rows) for c in range(cols)
        if board[r, c] == '.' and (r == rows - 1 or board[r + 1, c] != '.')
    )
    opponent_mobility = sum(
        1 for r in range(rows) for c in range(cols)
        if board[r, c] == '.' and (r == rows - 1 or board[r + 1, c] != '.')
    )

    # Final heuristic score
    w1, w2, w3, w4 = 1000, 500, 10, 1  # Weights for different components.
    score = (w1 * (player_threats - opponent_threats) +
             w2 * (player_mobility - opponent_mobility) +
             w3 * center_score)

    return score

# Example usage
board = np.array([
    ['.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', 'B', 'W', '.', '.', '.'],
    ['.', 'B', 'W', 'W', '.', '.', '.'],
    ['B', 'W', 'W', 'B', '.', '.', '.']
])

player = 'B'
score = evaluate_state_connect4(board, player)
print(f"Heuristic score for player {player}: {score}")


Heuristic score for player B: -10


In [3]:
import numpy as np

def evaluate_state_connect4(board, player):
    """
    Evaluate the state of a Connect 4 board for a given player.

    Parameters:
    - board: 2D numpy array representing the board. Each cell contains 'W' (white), 'B' (black), or '.' (empty).
    - player: The player for whom the evaluation is being calculated ('W' or 'B').

    Returns:
    - A heuristic score for the board state.
    """
    opponent = 'W' if player == 'B' else 'B'
    rows, cols = board.shape

    def count_patterns(length, target):
        """Count all patterns of the specified length for the given target player."""
        count = 0
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]

        for r in range(rows):
            for c in range(cols):
                for dr, dc in directions:
                    valid_pattern = True
                    target_count = 0

                    for step in range(length):
                        nr, nc = r + dr * step, c + dc * step
                        if 0 <= nr < rows and 0 <= nc < cols:
                            if board[nr, nc] == target:
                                target_count += 1
                            elif board[nr, nc] != '.':
                                valid_pattern = False
                                break
                        else:
                            valid_pattern = False
                            break

                    if valid_pattern and target_count == length:
                        count += 1

        return count

    def count_near_wins(target):
        """Count patterns where the target player has three tokens and one empty space."""
        count = 0
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]

        for r in range(rows):
            for c in range(cols):
                for dr, dc in directions:
                    valid_pattern = True
                    target_count = 0
                    empty_count = 0

                    for step in range(4):
                        nr, nc = r + dr * step, c + dc * step
                        if 0 <= nr < rows and 0 <= nc < cols:
                            if board[nr, nc] == target:
                                target_count += 1
                            elif board[nr, nc] == '.':
                                empty_count += 1
                            else:
                                valid_pattern = False
                                break
                        else:
                            valid_pattern = False
                            break

                    if valid_pattern and target_count == 3 and empty_count == 1:
                        count += 1

        return count

    def evaluate_center():
        """Calculate the advantage of controlling the center column."""
        center_column = cols // 2
        player_count = sum(1 for r in range(rows) if board[r, center_column] == player)
        opponent_count = sum(1 for r in range(rows) if board[r, center_column] == opponent)
        return player_count - opponent_count

    def evaluate_mobility():
        """Calculate mobility as the number of accessible empty cells for both players."""
        def count_accessible(target):
            return sum(
                1
                for r in range(rows)
                for c in range(cols)
                if board[r, c] == '.' and (r == rows - 1 or board[r + 1, c] != '.')
            )

        player_mobility = count_accessible(player)
        opponent_mobility = count_accessible(opponent)
        return player_mobility - opponent_mobility

    # Winning patterns
    if count_patterns(4, player) > 0:
        return float('inf')
    if count_patterns(4, opponent) > 0:
        return float('-inf')

    # Calculate scores for other factors
    threats_score = 1000 * (count_near_wins(player) - count_near_wins(opponent))
    center_score = 10 * evaluate_center()
    mobility_score = 500 * evaluate_mobility()

    return threats_score + center_score + mobility_score



In [4]:
# Example usage
board = np.array([
    ['.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', 'B', 'W', '.', '.', '.'],
    ['.', 'B', 'W', 'W', '.', '.', '.'],
    ['B', 'W', 'W', 'B', '.', '.', '.']
])

player = 'B'
score = evaluate_state_connect4(board, player)
print(f"Heuristic score for player {player}: {score}")


Heuristic score for player B: -10
